In [1]:
import os
import pickle
from tqdm import tqdm
from math import log
from nltk import PorterStemmer

In [2]:
folder = 'documents/'

In [3]:
k = 2
b = 0.75

stemmer = PorterStemmer()

N = 0
sum_len = 0

dl = {}
tf = {}
bm = {}

for d in tqdm(os.listdir(folder)):
    with open(folder + d) as f:
        dl[d] = 0
        for line in f.readlines():
            terms = [stemmer.stem(word).encode("utf8") for word in line.lower().split(' ') if len(word) > 1]
            dl[d] += len(terms)
            for t in terms:
                if t in bm:
                    if d not in bm[t]:
                        bm[t].append(d)
                else:
                    bm[t] = [d]
                if (t, d) in tf:
                    tf[(t, d)] += 1
                else:
                    tf[(t, d)] = 1
        N += 1
        sum_len += dl[d]
    
dl_ave = sum_len / N

100%|██████████| 1454/1454 [04:38<00:00,  5.23it/s]


In [4]:
dl_ave

5985.940165061898

In [5]:
N

1454

In [6]:
len(bm.keys())

35420

In [7]:
list(tf.keys())[:10]

[(b'dictionari',
  'document_from_url_with_hash_210eeef7d1d063712f8303e1087cc350'),
 (b'thesauru', 'document_from_url_with_hash_210eeef7d1d063712f8303e1087cc350'),
 (b'grammar', 'document_from_url_with_hash_210eeef7d1d063712f8303e1087cc350'),
 (b'a-z', 'document_from_url_with_hash_210eeef7d1d063712f8303e1087cc350'),
 (b'spell', 'document_from_url_with_hash_210eeef7d1d063712f8303e1087cc350'),
 (b'punctuat', 'document_from_url_with_hash_210eeef7d1d063712f8303e1087cc350'),
 (b'write', 'document_from_url_with_hash_210eeef7d1d063712f8303e1087cc350'),
 (b'help', 'document_from_url_with_hash_210eeef7d1d063712f8303e1087cc350'),
 (b'usag', 'document_from_url_with_hash_210eeef7d1d063712f8303e1087cc350'),
 (b'explor', 'document_from_url_with_hash_210eeef7d1d063712f8303e1087cc350')]

In [8]:
for t in tqdm(bm):
    scores = []
    df_t = len(bm[t])
    for d in bm[t]:
        scores.append(log(N / df_t) * ((k + 1) * tf[(t, d)]) / (k * (1 - b + b * dl[d] / dl_ave) + tf[(t, d)]))
    bm[t] = list(zip(scores, bm[t]))

100%|██████████| 35420/35420 [00:00<00:00, 45185.54it/s]


In [9]:
with open('bm.pickle', 'wb') as bm_file:
    pickle.dump(bm, bm_file)

In [10]:
max_len = 0
avg_len = 0

for t in tqdm(bm):
    max_len = max(max_len, len(bm[t]))
    avg_len += len(bm[t])
    
avg_len /= len(bm.keys())

100%|██████████| 35420/35420 [00:00<00:00, 637715.69it/s]


In [11]:
avg_len, max_len

(9.555957086391869, 1155)